calculate how many group, the number of vulnerabilities under every group, the total statistics of vuls --- serve for deployment of detection

collect the information about:

    - pkg_type: the name of ecosystem

    - location: whether the malicious code lies in

    - function: what is the function of malicious code

    - attack_type: how to define this type of attack in the scope of supply chain

    - trigger_mechanism: how the malicious code is triggerred

    - hide: how the malicious code hides itself to evade detection

    - iocs: the indicitors of attacks


In [1]:
from pathlib import Path
import pandas as pd
import json

In [2]:
# define process for npm
def load_json_file(json_file: Path) -> json:
    with json_file.open("r") as fr:
        return json.load(fr)
    
def type_check(json_file: Path) -> str:
    with json_file.open("r") as fr:
        aff = json.load(fr)['affected'][0]
        ecosystem = aff['package']['ecosystem']
    pkg_type = ""
    if "PyPI" in ecosystem:
        pkg_type = 'python'
    elif "crates" in ecosystem:
        pkg_type = 'rust'
    elif "npm" in ecosystem:
        pkg_type = 'npm'
    elif "RubyGems" in ecosystem:
        pkg_type = 'ruby'
    else:
        print("{} has not been included".format(ecosystem))

    return pkg_type

In [3]:

def npm_json_process(json_file: Path, ecosystem):
    data = load_json_file(json_file)
    pkg_type = ecosystem
    desc = data['details']
    try:
        iocs = data["database_specific"]["iocs"]
    except:
        print("The iocs are not included")
        iocs = None
        
    if "fully compromised" in desc:
        attack_type="data exfiltration/root shell"
        function='install powerful malware'
        trigger_mechanism = "download or upon installation"
        location="entrypoint/download"
        hide='N/A'
    
    elif "communicates" in desc and "domain" in desc:
        attack_type="C2 communication" 
        function='build communication tunnel with C2'
        trigger_mechanism="running network connection"
        location='socket or other communication protocols modules'
        hide="N/A"
    
    elif "blockchain" in desc and "cryptocurrency" in desc:
        attack_type="social engineering/typosquatting" 
        function='multi-stage malware download'
        trigger_mechanism="execute upon installation/asynchronous operations"
        location='functions'
        hide="split payloads/encoding"
    
    elif "blockchain" in desc and "cryptocurrency" in desc:
        attack_type="social engineering/typosquatting" 
        function='multi-stage malware download'
        trigger_mechanism="execute upon installation/asyncchronous operations"
        location='functions'
        hide="split payloads/encoding"

    elif "source code" in desc and "secrets" in desc:
        attack_type="" 
        function="steal source code and secrets"
        trigger_mechanism="execute upon installation"
        location="three inter hooked files"
        hide="split payloads"

    elif "commands" in desc and "malicious behavior" in desc:
        attack_type="execute malicious commands"
        function="N/A"
        trigger_mechanism="N/A"
        location="N/A"
        hide="N/A"
    
    elif "spyware" in desc and "information" in desc:
        attack_type="dropper/malware"
        function="spyware and information stealing"
        hide="base64-encoded"
        trigger_mechanism="execute automatically upon download"
        location="setup"
    
    elif "spyware" in desc and "information" in desc:
        attack_type="dropper/malware"
        function="spyware and information stealing"
        hide="base64-encoded"
        trigger_mechanism="execute automatically upon download"
        location="setup"

    elif "cloud storage" in desc and "binary executables" in desc:
        attack_type="N/A"
        function="information gathering"
        hide="N/A"
        trigger_mechanism="N/A"
        location="N/A"

    else:
        print("There is no process for this malicious package")
        print(desc)
        print(json_file)
        
    return pkg_type, location, function, attack_type, trigger_mechanism, hide, iocs


# define process for ruby
def ruby_json_process(json_file: Path, ecosystem):
    data = load_json_file(json_file)
    pkg_type = ecosystem
    desc = data['details']
    try:
        iocs = data["database_specific"]["iocs"]
    except:
        print("The iocs are not included")
        iocs = None
    if 'communicate' in desc and "domain" in desc:
        attack_type="C2 communication"
        function='build communication tunnel with C2'
        trigger_mechanism="network connection"
        location="socket or other communication protocols modules"
    else:
        print("There is no process for this malicious package")
        print(desc)
        print(json_file)
    hide="N/A"
    return pkg_type, location, function, attack_type, trigger_mechanism, hide, iocs


# define process for rust
def rust_json_process(json_file: Path, ecosystem):
    data = load_json_file(json_file)
    pkg_type = ecosystem
    desc = data['details']
    try:
        iocs = data["database_specific"]["iocs"]
    except:
        print("The iocs are not included")
        iocs = None
    if 'typosquat' in desc and "malware" in desc:
        # malware inside description --- typosquatting
        attack_type="typosquatting/malware"
        function='download payload'
    elif "communicate" in desc and "domain" in desc:
        attack_type="C2 communication" 
        function='build communication tunnel with C2'
    else:
        print("There is no process for this malicious package")
        print(desc)
        print(json_file)

    if data['id'] == "MAL-2022-1":
        trigger_mechanism="function call/condition/variable configuration"
        location='specific function'

    if data['id'] == "MAL-2023-8429":
        trigger_mechanism="running network connection"
        location='socket or other communication protocols modules'
    hide='N/A'
    return pkg_type, location, function, attack_type, trigger_mechanism, hide, iocs

# define process for public articles


In [4]:
# define process for pypi
def pypi_json_process(json_file: Path, ecosystem):
    data = load_json_file(json_file)
    pkg_type = ecosystem
    desc = data["details"]
    try:
        iocs = data["database_specific"]["iocs"]
    except:
        print("The iocs are not included")
        iocs = None

    if "extension" in desc and "crypto" in desc and "clipboard" in desc:
        attack_type="typosquatting"
        function="access the clipboard, replay crypto wallet address"
        hide="obfuscation"
        trigger_mechanism="installation"
        location="variable names in Chinese"

    elif "spyware" in desc and "information" in desc:
        attack_type="dropper/malware"
        function="spyware and information stealing"
        hide="base64-encoded"
        trigger_mechanism="execute automatically upon download"
        location="setup"

    elif "Alibaba" in desc and "AWS" in desc:
        attack_type="typosquatting/starjacking"
        function="steal sensitive credentials"
        hide="base64-encoded/embedded in functions"
        trigger_mechanism="functions called/send message"
        location="functions"

    elif "communicates" in desc and "domain" in desc:
        attack_type="C2 communication" 
        function='build communication tunnel with C2'
        trigger_mechanism="running network connection"
        location='socket or other communication protocols modules'
        hide="N/A"

    elif "Steganography" in desc:
        attack_type="typosquatting" 
        function='gain persistence/exfiltrate host information'
        trigger_mechanism="execute automatically upon download"
        location='setup'
        hide="steganography/obfuscation"

    elif "Check Point" in desc:
        attack_type="typosquatting"
        function="information steal/local passwords"
        trigger_mechanism="execute automatically upon download"
        location="setup"
        hide="N/A"
    
    elif "requests" in desc:
        attack_type="typosquatting/starjacking"
        function="download an open source cryptominer/subprocess/steal host information"
        trigger_mechanism="launch upon usage"
        location="requests function"
        hide="N/A"
    
    elif "commands" in desc and "malicious behavior" in desc:
        attack_type="execute malicious commands"
        function="N/A"
        trigger_mechanism="N/A"
        location="N/A"
        hide="N/A"

    else:
        print("There is no process for this malicious package")
        print(desc)
        print(json_file)
        
    return pkg_type, location, function, attack_type, trigger_mechanism, hide, iocs

In [5]:
# iterate the json under per folder
data_folder = Path.cwd().parent.joinpath("data","malicious")

data_metrics = {
    "pkg_type": [],
    "location": [],
    "function": [],
    "attack_type":[],
    "trigger_mechanism":[],
    "hide":[]
}

ioc_dict = {
    "domains":[],
    "ips":[],
    "links":[]
}

for number, json_file in enumerate(data_folder.rglob("**/*.json")):
    ecosystem = json_file.parents[1].name
    if ecosystem == 'npm':
        results = npm_json_process(json_file, ecosystem)
    elif ecosystem == 'crates-io':
        results = rust_json_process(json_file, 'rust')
    elif ecosystem == "pypi":
        results = pypi_json_process(json_file, ecosystem)
    elif ecosystem == 'rubygems':
        results = ruby_json_process(json_file, 'ruby')
    else:
        print("there is no available package process for {}".format(ecosystem))
        
    data_metrics['pkg_type'].append(results[0])
    data_metrics['location'].append(results[1])
    data_metrics['function'].append(results[2])
    data_metrics['attack_type'].append(results[3])
    data_metrics['trigger_mechanism'].append(results[4])
    data_metrics['hide'].append(results[5])

    if results[6]:
        for key, value in results[6].items():
            if key in ioc_dict.keys():
                ioc_dict[key].extend(results[6][key])
            else:
                ioc_dict[key] = results[6][key]


The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are not included
The iocs are

In [6]:
data_metrics

{'pkg_type': ['pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi',
  'pypi'

In [7]:
import pandas as pd

In [8]:
df_fea = pd.DataFrame(data_metrics)
df_fea.to_csv("../data/stas/data_metrics.csv", index=False)

In [56]:
ioc_dict

{'domains': ['serene-springs-50769.herokuapp.com',
  'serene-springs-50769.herokuapp.com',
  'serene-springs-50769.herokuapp.com',
  'serene-springs-50769.herokuapp.com',
  'serene-springs-50769.herokuapp.com',
  'serene-springs-50769.herokuapp.com',
  'serene-springs-50769.herokuapp.com',
  'serene-springs-50769.herokuapp.com',
  'serene-springs-50769.herokuapp.com',
  'serene-springs-50769.herokuapp.com',
  'cryptopriceoffer.com',
  'npmjscloud.com',
  'npmrepos.com',
  'tradingprice.net',
  'npmjsregister.com',
  'npmcloudjs.com',
  'bi2price.com',
  'npmaudit.com',
  'coingeckoprice.com',
  '6wxd3v84nevku06dcgbqcxrmt.canarytokens.com',
  'fhg62xavat9jzyt6euwxi6sro.canarytokens.com',
  '1wy3rk316x8qqy4fyxtvcs4kkbq2es2h.oastify.com',
  '288utkkrohmp0nr8znflcp88nztrhg.oastify.com',
  'bq5m9lnmalh9ktyi9wydockt9kfb32rr.oastify.com',
  'c7kxnys58daceezcxx0jjstn6ec50vok.oastify.com',
  'cczk46g2vtc0000k68dgggx31deyyyyyb.oast.fun',
  'cfrg38n2vtc0000h72xgg8hebweyyyyyb.oast.fun',
  'cfswk0m

In [9]:
# Find the maximum length among the values in the dictionary
max_length = max(len(value) if isinstance(value, list) else 1 for value in ioc_dict.values())

# Pad the lists to the maximum length
for key, value in ioc_dict.items():
    if isinstance(value, list) and len(value) < max_length:
        ioc_dict[key] += [None] * (max_length - len(value))


df_iocs = pd.DataFrame(ioc_dict)
df_iocs.to_csv("../data/stas/data_iocs.csv", index=False)